In [1]:
import requests
import re
import time
import urllib.request

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [2]:
class Item:
    """
    상품 상세페이지의 정보를 저장하는 클래스
    """
    def __init__(self):
        self.name = ""
        self.price = 0
        self.option = []
        self.bundle_shipping_item_page_URL = ""
        self.URL = ""
        self.shipping_charge = 0
        
    def setName(self,givenName):
        self.name = givenName
    def setPrice(self,givenPrice):
        self.price = givenPrice
    def setOption(self,givenOption):
        self.option = givenOption
    def setBundleItemPageURL(self,givenBundleURL):
        self.bundle_shipping_item_page_URL = givenBundleURL
    def setURL(self,givenURL):
        self.URL = givenURL
    def setShippingCharge(self,givenShippingCharge):
        self.shipping_charge = givenShippingCharge
        
    def getName(self):
        return self.name
    def getPrice(self):
        return self.price
    def getOption(self):
        return self.option
    def getBundleItemPageURL(self):
        return self.bundle_shipping_item_page_URL
    def getURL(self):
        return self.URL
    def getShippingCharge(self):
        return self.shipping_charge
        
    def __str__(self):
        msg = "이름: {}\n가격: {}\n배송비: {}\n옵션정보: {}\n묶음배송상품정보URL: {}\n상품URL: {}".format(self.name,self.price,self.shipping_charge,self.option,self.bundle_shipping_item_page_URL,self.URL)
        return msg

In [3]:
def scroll_down_page():
    """
    (void)->void
    현재 구동중인 크롬 드라이버 페이지를 스크롤 다운 하는 함수
    """
    SCROLL_PAUSE_SEC = 0.5
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 500만큼 스크롤해서 끝까지 스크롤 다운
        scrollY = driver.execute_script("return window.scrollY")
        driver.execute_script("window.scrollTo(0, window.scrollY + 300);")
        new_scrollY = driver.execute_script("return window.scrollY")

        while new_scrollY != scrollY:
            scrollY = new_scrollY
            driver.execute_script("window.scrollTo(0, window.scrollY + 500);")
            new_scrollY = driver.execute_script("return window.scrollY")

        new_height = driver.execute_script("return document.body.scrollHeight")
        # 0.5초 대기
        if(last_height == new_height):
            time.sleep(SCROLL_PAUSE_SEC)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if(last_height == new_height):
                break

        last_height = new_height

In [4]:
def move_next_page_naver_shopping_search():
    """
    (void) -> void
    네이버 쇼핑 검색 사이트에서 다음 페이지를 불러오는 함수
    """
    driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[3]/div[1]/div[3]/div/span//following::a").click()

In [5]:
def find_item_info_from_naver_shopping_search():
    """
    (void) -> list of Item object
    네이버 쇼핑 검색을 통해 상품에 대한 정보를 Item 객체 리스트로 만들어서 반환
    """
    req = driver.page_source
    soup = BeautifulSoup(req, "lxml")
    
    item_list = []
    item_containers = soup.find_all("li", attrs={"class":"basicList_item__2XT81"})
    for item_info in item_containers:
        # 광고상품 제외
        if(item_info.find("button", attrs={"class":"ad_ad_stk__12U34"})):
            continue
        
        # 쇼핑몰별로 가격정보가 적힌 요소
        if(item_info.find("span", attrs={"class":"price_low__2vp2A"})):
            url = item_info.find("a", attrs={"class":"basicList_link__1MaTN"})["href"]
            headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
            req = requests.get(url, headers)
            new_soup = BeautifulSoup(req.text, "lxml")    
            
            sub_page_item_list = new_soup.find("ul",attrs={"class":"productList_list_seller___NsJk"}).find_all("li")
            for sub_item in sub_page_item_list:
                item = Item()
                item_name = sub_item.find("a",attrs={"class":"productList_title__uCZ0P"}).get_text()
                item.setName(item_name)
                
                item_url = (sub_item.find("a",attrs={"class":"productList_title__uCZ0P"})["href"])
                item.setURL(item_url)
                
                item_price = (int(sub_item.find("div", attrs={"class":"productList_price__2FKhU"})
                                           .find("em")
                                           .get_text()
                                           .replace(",","")))
                item.setPrice(item_price)
                
                item_shipping_charge = (int(sub_item.find("div",attrs={"class":"productList_delivery__2BusJ"})
                                                   .get_text()
                                                   .replace(",","")
                                                   .replace("원","")
                                                    .replace("무료배송","0")))
                item.setShippingCharge(item_shipping_charge)
                item_list.append(item)
        # 한 쇼핑몰에 대한 가격정보가 적힌 요소
        else:
            item = Item()
            item_name = item_info.find("a", attrs={"class":"basicList_link__1MaTN"}).get_text()
            item.setName(item_name)
            
            item_url = (item_info.find("a", attrs={"class":"basicList_link__1MaTN"}).attrs["href"])
            item.setURL(item_url)
            
            item_price = (int(item_info.find("span", attrs={"class":"price_num__2WUXn"})
                             .get_text()
                             .replace(",","")
                             .replace("원",""))) 
            item.setPrice(item_price)
            
            item_shipping_charge = (int(item_info.find("em", attrs={"class":"basicList_option__3eF2s"})
                                       .get_text()
                                       .replace("배송비 무료","0")
                                       .replace("배송비","")
                                       .replace("원","")
                                       .replace(",","")))
            item.setShippingCharge(item_shipping_charge)
            
            item_list.append(item)
    return item_list

In [6]:
def check_exists_by_class_name(driver, class_name):
    """
    (driver object, string)->bool
    selenium driver를 사용해서 해당 페이지에 class attribute name이 일치하는 element가 있는지 확인하는 함수
    """
    try:
        driver.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

In [7]:
def find_item_info_from_naver_smartstore(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 스마트스토어 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, '_3R5zw_nN-K')):
        driver.find_element_by_class_name('_3R5zw_nN-K').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("a", attrs={"class":"_3R5zw_nN-K"})):
        option_list = []
        option_list_code = soup.find_all("a", attrs={"class":"_3cJFxK4gO7"})
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
    if(soup.find("a", attrs={"class":"_2Ugyq5gjSq _23ZcKYMJAs _2W6umRasi3 N=a:pcs.delivery"})):
        bundle_item_URL = soup.find("a", attrs={"class":"_2Ugyq5gjSq _23ZcKYMJAs _2W6umRasi3 N=a:pcs.delivery"})["href"]
        item.setBundleItemPageURL("https://smartstore.naver.com" + bundle_item_URL)



def find_item_info_from_auction(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 옥션 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'item_options opt_group uxeselectbox')):
        driver.find_element_by_class_name('item_options opt_group uxeselectbox').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":"select-itemoption-list uxeselect_dropdown Group_Item_List_Wrapper on"})):
        option_list = []
        option_list_code = soup.find("ul", attrs={"class":"select-itemoption-list uxeselect_dropdown Group_Item_List_Wrapper on"}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
    if(soup.find("a", attrs={"class":"btn_gostore"})):
        bundle_item_URL = soup.find("a", attrs={"class":"btn_gostore"})["href"]
        item.setBundleItemPageURL(bundle_item_URL)
    
    
def find_item_info_from_gmarket(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 지마켓 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'select-item_option uxeselect_btn')):
        driver.find_element_by_class_name('select-item_option uxeselect_btn').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":"select-itemoption-list type_minishop uxeselect_dropdown on"})):
        option_list = []
        option_list_code = soup.find("ul", attrs={"class":"select-itemoption-list type_minishop uxeselect_dropdown on"}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
    if(soup.find("a", attrs={"class":"_2Ugyq5gjSq _23ZcKYMJAs _2W6umRasi3 N=a:pcs.delivery"})):
        bundle_item_URL = soup.find("a", attrs={"class":"btn_gostore"})["href"]
        item.setBundleItemPageURL("bundle_item_URL")
    
    
    
def find_item_info_from_interpark(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 인터파크 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'optionSelect addSelected')):
        driver.find_element_by_class_name('optionSelect addSelected').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("div", attrs={"class":"options"})):
        option_list = []
        option_list_code = soup.find("div", attrs={"class":"options"}).find("ul").find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
    if(soup.find("a", attrs={"class":"arrow"})):
        bundle_item_URL = soup.find("a", attrs={"class":"arrow"})["href"]
        item.setBundleItemPageURL(bundle_item_URL)
    
    
    
def find_item_info_from_tmon(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 티몬 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'tit')):
        driver.find_element_by_class_name('tit').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":"purchase_selector"})):
        option_list = []
        option_list_code = soup.find("ul", attrs={"class":"purchase_selector"}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
    
    
def find_item_info_from_lotteon(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 롯데온라인쇼핑몰 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'selectResult')):
        driver.find_element_by_class_name('selectResult').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":re.compile("^select-pageOpt")})):
        option_list = []
        option_list_code = soup.find_all("ul", attrs={"class":re.compile("^select-pageOpt")}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)

        
    
def find_item_info_from_wemakeprice(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 위메프 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'item_selbox')):
        driver.find_element_by_class_name('item_selbox').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":"item_option"})):
        option_list = []
        option_list_code = soup.find("ul", attrs={"class":"item_option"}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
# 위메프는 묶음배송 상품정보가 상품 상세페이지에 있기 때문에 따로 묶음배송 상품정보 URL을 입력할 필요 없음.   
    if(soup.find("a", attrs={"data-gtm-label":"묶음배송상품보기"})):
        item.setBundleItemPageURL(item.getURL())
    
    
    
def find_item_info_from_11st(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 11번가 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'btn_option dropdown_selected')):
        driver.find_element_by_class_name('btn_option dropdown_selected').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":"option_item_list b_product_buy_option"})):
        option_list = []
        option_list_code = soup.find("ul", attrs={"class":"option_item_list b_product_buy_option"}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)
        
    if(soup.find("div", attrs={"class":"c_product_seller_btn"})):
        bundle_item_URL = soup.find("div", attrs={"class":"c_product_seller_btn"}).find("button")["onclick"].replace("window.open('","").replace("')","")
        item.setBundleItemPageURL(bundle_item_URL)
    
    
    
def find_item_info_from_coupang(drive, item):
    """
    (Item object) -> void
    인자로 받은 item객체의 option정보와 묶음배송상품정보URL을 쿠팡 상세 페이지에서 가져오는 함수
    """
    if(check_exists_by_class_name(driver, 'prod-option__selected  multiple')):
        driver.find_element_by_class_name('prod-option__selected  multiple').click()
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    
    if(soup.find("ul", attrs={"class":"prod-option__list"})):
        option_list = []
        option_list_code = soup.find("ul", attrs={"class":"prod-option__list"}).find_all("li")
        for option_code in option_list_code:
            option_list.append(option_code.get_text())
        item.setOption(option_list)

In [8]:
def find_bundle_shipping_item_info_from_naver_smartstore(driver, new_recommanded_purchase_list, non_bundled_purchase_list):
    """
    (driver(), {"string":Item()}, {"string":Item()}) -> void
    네이버 스마트스토어 묶음배송 상품 정보에서 다른 상품의 정보를 받아서 new_recommanded_purchase_list에 저장
    """
    
    new_purchase_item_list = {}
    
    for search_keyword in search_keyword_list:
        if(search_keyword in new_recommanded_purchase_list.keys()):
            continue
        item_list = []
        
        search_box = driver.find_element_by_class_name("_38DpYZHzTy")
        time.sleep(0.5)
        search_box.clear()
        time.sleep(0.5)
        search_box.send_keys(search_keyword)
        time.sleep(0.5)
        search_box.send_keys(Keys.RETURN)
        
        req = driver.page_source
        soup = BeautifulSoup(req, 'lxml')
        if(soup.find("ul", attrs={"class":"_3IAxB5KmaQ"})):
            item_container_list = soup.find("ul", attrs={"class":"_3IAxB5KmaQ"}).find_all("li")
            
            for item_container in item_container_list:
                item = Item()
                item.setURL("https://smartstore.naver.com" + item_container.find("a", attrs={"class":"cj7gkLIEbC N=a:lst.product linkAnchor"})["href"])
                item.setName(item_container.find("strong", attrs={"class":"_3W4A_RFJRS"}).get_text())
                item.setPrice(int(item_container.find("span", attrs={"class":"_11N87Svo1h"})
                             .get_text()
                             .replace("원","")
                             .replace(",","")))
                item_list.append(item)
                
            new_purchase_item_list[search_keyword] = item_list       
    for search_keyword in new_purchase_item_list.keys():
        minimum_price_item = find_minimum_price_item(new_purchase_item_list[search_keyword])
        if(compare_total_price(non_bundled_purchase_list[search_keyword], minimum_price_item) == 1):
            new_recommanded_purchase_list[search_keyword] = minimum_price_item
        
    for search_keyword in non_bundled_purchase_list.keys():
        if(search_keyword not in new_recommanded_purchase_list):
            new_recommanded_purchase_list[search_keyword] = non_bundled_purchase_list[search_keyword]

            
            
def compare_total_price(item1, item2):
    """
    (Item(),Item())->int
    두 Item객체의 배송비와 가격의 합을 비교하는 함수
    """
    item1_total_price = item1.getPrice() + item1.getShippingCharge()
    item2_total_price = item2.getPrice() + item2.getShippingCharge()
    if(item1_total_price > item2_total_price):
        return 1
    elif(item1_total_price < item2_total_price):
        return -1
    else:
        return 0

In [9]:
def find_minimum_price_item(item_list):
    """
    (list of Item())->Item()
    주어진 Item객체 list에서 가격과 배송비의 합이 가장 작은 Item객체를 반환하는 함수
    """
    min_total_price = item_list[0].getPrice() + item_list[0].getShippingCharge()
    min_price_item = item_list[0]
    
    for item in item_list:
        total_price = item.getPrice() + item.getShippingCharge()
        if(total_price < min_total_price):
            min_total_price = total_price
            min_price_item = item
            
    return min_price_item



def calc_total_price(recommand_list):
    """
    ({"string":Item()}) -> bool
    추천 구매 리스트의 구매 금액 총합을 반환
    """
    total_price = 0
    
    for item in recommand_list.values():
        total_price += (item.getPrice() + item.getShippingCharge())
        
    return total_price


    
def recommand_best_purchasing(purchase_candidate_list):
    """
    ({string:list[Item()]}) -> list[{string:Item()}]
    주어진 Item의 list에서 각 상품별로 가장 싼 제품을 택하는 선택지 이외에,
    묶음배송상품을 고려하였을 때, 더 저렴하게 살 수 있는 선택지들을 list로 만들어 반환하는 함수
    """
    non_bundled_purchase_list = {}
    
    for search_keyword in search_keyword_list:
        non_bundled_purchase_list[search_keyword] = find_minimum_price_item(purchase_candidate_list[search_keyword])
        
    path = "C:/Users/82106/Downloads/chromedriver.exe"
    driver = webdriver.Chrome(path)
    recommand_purchase_list_candidate = []
    for search_keyword in search_keyword_list:
        for item in purchase_candidate_list[search_keyword]:
            URL = item.getBundleItemPageURL()
            if(URL != ""):
                driver.get(URL)
                new_recommanded_purchase_list = {}
                new_recommanded_purchase_list[search_keyword] = item
                if("smartstore" in driver.current_url):
                    find_bundle_shipping_item_info_from_naver_smartstore(driver, new_recommanded_purchase_list, non_bundled_purchase_list)
                    if(calc_total_price(new_recommanded_purchase_list) < calc_total_price(non_bundled_purchase_list)):
                        recommand_purchase_list_candidate.append(new_recommanded_purchase_list)
    
    return recommand_purchase_list_candidate

In [15]:
# 찾고자 하는 상품명들을 입력 후, 검색 결과를 가져옴
path = "C:/Users/82106/Downloads/chromedriver.exe"
driver = webdriver.Chrome(path)
search_keyword_list = []
while(True):
    search_keyword = input(prompt = "찾을 품목의 정확한 이름을 입력하세요.(종료하려면 0) : ")
    if(search_keyword == "0"):
        break
    search_keyword_list.append(search_keyword)
    
search_item_list = {}

for search_keyword in search_keyword_list:
    driver.get("https://shopping.naver.com/")
    search_box = driver.find_element_by_name("query")
    time.sleep(0.5)
    search_box.send_keys(search_keyword)
    search_box.send_keys(Keys.RETURN)
    
    scroll_down_page()
    search_item_list[search_keyword] = find_item_info_from_naver_shopping_search()


찾을 품목의 정확한 이름을 입력하세요.(종료하려면 0) : 신라면
찾을 품목의 정확한 이름을 입력하세요.(종료하려면 0) : 카레여왕
찾을 품목의 정확한 이름을 입력하세요.(종료하려면 0) : 햇반
찾을 품목의 정확한 이름을 입력하세요.(종료하려면 0) : 허니버터칩
찾을 품목의 정확한 이름을 입력하세요.(종료하려면 0) : 0


In [16]:
for key in search_item_list.keys():
    for item in search_item_list[key]:
        print(item)

이름: 농심 신라면 봉지 120g
가격: 490
배송비: 3000
옵션정보: []
묶음배송상품정보URL: 
상품URL: https://cr.shopping.naver.com/adcr.nhn?x=cbUFcOMbrug0L1XW1ox2Sv%2F%2F%2Fw%3D%3DshDeV5rrR7ODchI7d2UCwEsk14rQVQP2TMQKPsFAR027sxnCeUdoJKwdm6dccG%2B8J9yl64KujWgSA5ECNazEQYXMpUAAE9o1LR%2Bej1x8D5WubtBWaC5hWhwN7Bw1I2yL%2B0v1KD%2F5EtY161pvk5jILPkS%2BortKzfQDWBF5GV%2FKfqEhJv3ljJo79VVIpj6%2Fm3WO4g5cmqoNway0KGtbsWWa8M2HYWYcQlZP02BVDdc68f1WEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2MszRsUfNrHrMFzKVIupmN94NxjinRYmpw5wc6R0bTuHqNn6Uiu22eOMpZmBoDD5tTa8z6W%2BL084AhZ0WquZHS%2BaJzp6k6d%2FmX2FIAug3P8irXXw%2Byg0cs9wsP3ZQy5x0mym4TEQPhrcCSdiSwqFMbTCLx6sB9UXluBYtA0JpFp6%2BU%2F3%2FM98dCBsmwBMraIHUHhlOzFskaCyn0%2FpeDgLpbwAUCbq93rnvxiWBHn8P1Y2k2oJDXx9uDahl9WuMZaPEYZSPjR5%2BPfZ1n55jLIm1GYGf8msIlx3X6IrdTpGZ2GunTBFWmC%2Fe%2BUWOZkhWtfc2dO70DqsmGdJT9uS76snzkOY%3D&nvMid=82396792200&catId=50002385
이름: 농심 신라면 120g x 1개
가격: 490
배송비: 3000
옵션정보: []
묶음배송상품정보URL: 
상품URL: https://cr.shopping.naver.com/adcr.nhn?x=xhW0np0LUfNsqwf0VvApaf%2F%2F%2Fw%3D%3Ds14r93w

In [17]:
# 검색된 상품 중 일부 상품의 상세페이지에 입장해서 상세정보 조회
input_num = int(input(prompt="정보를 볼 상품의 수를 입력하세요: "))

for search_keyword in search_keyword_list:
    for idx in range(0,input_num):
        URL = search_item_list[search_keyword][idx].getURL()
        item = search_item_list[search_keyword][idx]
        driver.get(URL)
        if("smartstore" in driver.current_url):
            find_item_info_from_naver_smartstore(driver, item)
#         elif("auction" in drive.current_url):
#             find_item_info_from_auction(driver, item)
#         elif("gmarket" in drive.current_url):
#             find_item_info_from_gmarket(driver, item)
#         elif("interpark" in drive.current_url):
#             find_item_info_from_interpark(driver, item)
#         elif("tmon" in drive.current_url):
#             find_item_info_from_tmon(driver, item)
#         elif("lotteon" in drive.current_url):
#             find_item_info_from_lotteon(driver, item)
#         elif("wemakeprice" in drive.current_url):
#             find_item_info_from_wemakeprice(driver, item)
#         elif("11st" in drive.current_url):
#             find_item_info_from_11st(driver, item)
#         elif("coupang" in drive.current_url):
#             find_item_info_from_coupang(driver, item) # 미구현 부분이라 주석처리


정보를 볼 상품의 수를 입력하세요: 10


In [18]:
# 상품 정보를 조회한 상품들을 구매 결정 후보군으로 입력함
# 구매 결정 후보군의 묶음배송상품 정보를 조회한 후, 최종적으로 최저가로 구매 가능한 조합을 추천

purchase_candidate_list = {}
for search_keyword in search_keyword_list:
    purchase_candidate_list[search_keyword] = search_item_list[search_keyword][0:input_num]

recommanded_purchase_list_candidate = recommand_best_purchasing(purchase_candidate_list)

In [19]:
for key in search_item_list.keys():
    for item in purchase_candidate_list[key]:
        print(item)

이름: 농심 신라면 봉지 120g
가격: 490
배송비: 3000
옵션정보: ['농심 신라면 봉지 120g']
묶음배송상품정보URL: https://smartstore.naver.com/plan88/bundle/20904313?prevProdId=4852268901
상품URL: https://cr.shopping.naver.com/adcr.nhn?x=cbUFcOMbrug0L1XW1ox2Sv%2F%2F%2Fw%3D%3DshDeV5rrR7ODchI7d2UCwEsk14rQVQP2TMQKPsFAR027sxnCeUdoJKwdm6dccG%2B8J9yl64KujWgSA5ECNazEQYXMpUAAE9o1LR%2Bej1x8D5WubtBWaC5hWhwN7Bw1I2yL%2B0v1KD%2F5EtY161pvk5jILPkS%2BortKzfQDWBF5GV%2FKfqEhJv3ljJo79VVIpj6%2Fm3WO4g5cmqoNway0KGtbsWWa8M2HYWYcQlZP02BVDdc68f1WEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2MszRsUfNrHrMFzKVIupmN94NxjinRYmpw5wc6R0bTuHqNn6Uiu22eOMpZmBoDD5tTa8z6W%2BL084AhZ0WquZHS%2BaJzp6k6d%2FmX2FIAug3P8irXXw%2Byg0cs9wsP3ZQy5x0mym4TEQPhrcCSdiSwqFMbTCLx6sB9UXluBYtA0JpFp6%2BU%2F3%2FM98dCBsmwBMraIHUHhlOzFskaCyn0%2FpeDgLpbwAUCbq93rnvxiWBHn8P1Y2k2oJDXx9uDahl9WuMZaPEYZSPjR5%2BPfZ1n55jLIm1GYGf8msIlx3X6IrdTpGZ2GunTBFWmC%2Fe%2BUWOZkhWtfc2dO70DqsmGdJT9uS76snzkOY%3D&nvMid=82396792200&catId=50002385
이름: 농심 신라면 120g x 1개
가격: 490
배송비: 3000
옵션정보: ['신라면 120g x 1개', '올리

In [20]:
idx = 1
for recommanded_purchase_list in recommanded_purchase_list_candidate:
    print("list " ,idx)
    idx += 1
    total_price = 0
    for value in recommanded_purchase_list.values():
        print()
        print(value)
        total_price += value.getPrice() + value.getShippingCharge()
    print("가격 총합 : ", total_price)
    print()

list  1

이름: 농심 신라면 봉지 120g
가격: 490
배송비: 3000
옵션정보: ['농심 신라면 봉지 120g']
묶음배송상품정보URL: https://smartstore.naver.com/plan88/bundle/20904313?prevProdId=4852268901
상품URL: https://cr.shopping.naver.com/adcr.nhn?x=cbUFcOMbrug0L1XW1ox2Sv%2F%2F%2Fw%3D%3DshDeV5rrR7ODchI7d2UCwEsk14rQVQP2TMQKPsFAR027sxnCeUdoJKwdm6dccG%2B8J9yl64KujWgSA5ECNazEQYXMpUAAE9o1LR%2Bej1x8D5WubtBWaC5hWhwN7Bw1I2yL%2B0v1KD%2F5EtY161pvk5jILPkS%2BortKzfQDWBF5GV%2FKfqEhJv3ljJo79VVIpj6%2Fm3WO4g5cmqoNway0KGtbsWWa8M2HYWYcQlZP02BVDdc68f1WEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2MszRsUfNrHrMFzKVIupmN94NxjinRYmpw5wc6R0bTuHqNn6Uiu22eOMpZmBoDD5tTa8z6W%2BL084AhZ0WquZHS%2BaJzp6k6d%2FmX2FIAug3P8irXXw%2Byg0cs9wsP3ZQy5x0mym4TEQPhrcCSdiSwqFMbTCLx6sB9UXluBYtA0JpFp6%2BU%2F3%2FM98dCBsmwBMraIHUHhlOzFskaCyn0%2FpeDgLpbwAUCbq93rnvxiWBHn8P1Y2k2oJDXx9uDahl9WuMZaPEYZSPjR5%2BPfZ1n55jLIm1GYGf8msIlx3X6IrdTpGZ2GunTBFWmC%2Fe%2BUWOZkhWtfc2dO70DqsmGdJT9uS76snzkOY%3D&nvMid=82396792200&catId=50002385

이름: CJ 햇반 200g
가격: 710
배송비: 0
옵션정보: []
묶음배송상품정보URL: 
상품U